# DEM co-registration tutorial

When comparing two different elevation models, it is important to ensure that the elevation models are not shifted relative to each other. Small horizontal and vertical shifts between the two elevation models introduce biases that lead to inaccurate estimation of the differences between the two elevation models. Fortunately, by comparing the elevation differences with the slope and aspect of the points where the differences are measured, we can model and remove these shifts, leading to more accurate estimations of the differences between two elevation models. Further details can be found in [Nuth and Kääb (2011)](https://www.the-cryosphere.net/5/271/2011/tc-5-271-2011.html).

This walkthrough provides a step-by-step illustration of the method for co-registration, using [pybob](https://pybob.readthedocs.io). You can re-use this example with your own data, or use the scripts and functions provided by pybob (see the documentation for more info).

In [ ]:
%matplotlib notebook

# imports - have to read the data, run individual steps, and show the results
from pybob.GeoImg import GeoImg
import pybob.coreg_tools as ct
from pybob.plot_tools import set_pretty_fonts
import matplotlib.pyplot as plt
import numpy as np

plt.ion()
set_pretty_fonts(font_size=18)
np.seterr(all='ignore')

## Setup and initial inspection

Now that we have imported the necessary modules/packages, we can start by loading the two DEMs, reprojecting the DEMs to the same spatial reference system, extent, and cell size, and calculating slope and aspect to use in the co-registration process:

In [ ]:
master_dem = GeoImg('data/dem_25m_statkart.tif') # replace the filename with your own master DEM here
slave_dem = GeoImg('data/AST14DMO_00308122004104803_DEM.tif') # replace the filename with your own slave DEM here

mst_reproj = master_dem.reproject(slave_dem) # reproject master DEM to slave DEM extent, cell size

slope_geo = ct.get_slope(mst_reproj) # calculate slope from master DEM
aspect_geo = ct.get_aspect(mst_reproj) # calculate aspect from master DEM

We can now difference the two DEMs and display the difference in a figure. What do you notice about the difference between the DEMs? Are there any patterns that jump out?

In [ ]:
init_dh_geo = slave_dem.copy(new_raster=mst_reproj.img - slave_dem.img) # make a new dataset with the initial difference

# display the new difference image. for more info, see the documentation
fig1 = init_dh_geo.display(fig=plt.figure(figsize=(8,8)), cmap='RdYlBu', vmin=-20, vmax=20)
fig1.gca().set_xlabel('utm easting (m)')
fig1.gca().set_ylabel('utm northing (m)')

# add a colorbar and set the label
cb = plt.colorbar(); cb.set_label('elevation difference (m)')

## First iteration

Let's start the iterative co-registration process. First, we'll extract elevation change, slope, and aspect for a number of points in the overlap area, then plot the difference between the DEMs shown as a hillshade to emphasize differences on slopes.

In [ ]:
this_slave = slave_dem.copy() # make a copy of the slave DEM

dH, xdata, ydata, sdata = ct.preprocess(np.ones(mst_reproj.img.shape) == 0, slope_geo.img, aspect_geo.img, mst_reproj, this_slave)
fig3 = ct.false_hillshade(dH, 'DEM difference pre-coregistration')

# set the shift variables to be updated during this process.
x_shift = y_shift = z_shift = 0

xadj, yadj, zadj, fig2 = ct.coreg_fitting(xdata, ydata, sdata, 'Iteration 1')

Clearly, there are large offsets between the DEMs, as we noticed from the initial difference map. Based on this difference, we estimate a difference of approximately -42 meters in the x direction, 6 meters in the y direction, and 15 meters in the z direction. 

### Question: can you think of a reason why the difference in the z direction might be so high?


Now that we have initial estimates of the offsets in the x,y, and z directions, we can shift the slave raster and look at the difference again.

In [ ]:
x_shift += xadj
y_shift += yadj
z_shift += zadj

print(x_shift, y_shift, z_shift)
this_slave.shift(xadj, yadj)
this_slave = this_slave.reproject(mst_reproj) # re-align the grid of the master, slave DEMs after shifting

this_slave = this_slave.copy(new_raster=this_slave.img + zadj) # shift the DEM in the z direction

dH, xdata, ydata, sdata = ct.preprocess(np.ones(mst_reproj.img.shape) == 0, slope_geo.img, aspect_geo.img, mst_reproj, this_slave)
fig4 = ct.false_hillshade(dH, 'DEM difference after first iteration.')

## Second iteration

Now, we can estimate the new shift between the rasters, and apply the new shift.

In [ ]:
xadj, yadj, zadj, fig5 = ct.coreg_fitting(xdata, ydata, sdata, 'Iteration 2')

x_shift += xadj
y_shift += yadj
z_shift += zadj

print(x_shift, y_shift, z_shift)

this_slave.shift(xadj, yadj)

this_slave = this_slave.reproject(mst_reproj)
this_slave = this_slave.copy(new_raster=this_slave.img + zadj) # shift the DEM in the z direction

dH, xdata, ydata, sdata = ct.preprocess(np.ones(mst_reproj.img.shape) == 0, slope_geo.img, aspect_geo.img, mst_reproj, this_slave)
fig4 = ct.false_hillshade(dH, 'DEM difference after second iteration.')

## Third iteration

Now, we can estimate the new shift between the rasters, and apply the new shift.

In [ ]:
dH, xdata, ydata, sdata = ct.preprocess(np.ones(mst_reproj.img.shape) == 0, slope_geo.img, aspect_geo.img, mst_reproj, this_slave)
xadj, yadj, zadj, fig5 = ct.coreg_fitting(xdata, ydata, sdata, 'Iteration 3')

x_shift += xadj
y_shift += yadj
z_shift += zadj

print(x_shift, y_shift, z_shift)

this_slave.shift(xadj, yadj)

this_slave = this_slave.reproject(mst_reproj)
this_slave = this_slave.copy(new_raster=this_slave.img + zadj) # shift the DEM in the z direction

dH, xdata, ydata, sdata = ct.preprocess(np.ones(mst_reproj.img.shape) == 0, slope_geo.img, aspect_geo.img, mst_reproj, this_slave)
fig5 = ct.false_hillshade(dH, 'DEM difference after third iteration.')


So after three iterations, we're no longer seeing a significant pattern in the aspect vs. normalized dH plot, so we don't really need to do any more iterations. The final shift that we estimate between the DEMs is approximately -47 meters in the x direction, 7 meters in the y direction, and 15.4 meters in the z direction.

## Final plots and saving the results

Now, we can plot a histogram of the elevation difference distributions for the DEMs pre-coregistration (black) and post-coregistration (red), save the co-registered DEM, and re-display the difference map from before.

In [ ]:
ct.final_histogram(init_dh_geo.img, dH.img) # plot the difference distributions between the two DEMs pre- and post- co-registration.

this_slave.write('data/AST14DMO_00308122004104803_DEM_adj.tif') # save the shifted slave DEM

In [ ]:
final_dh_geo = slave_dem.copy(new_raster=mst_reproj.img - this_slave.img)

# display the new difference image. for more info, see the documentation
fig1 = final_dh_geo.display(fig=plt.figure(figsize=(8,8)), cmap='RdYlBu', vmin=-20, vmax=20)
fig1.gca().set_xlabel('utm easting (m)')
fig1.gca().set_ylabel('utm northing (m)')

# add a colorbar and set the label
cb = plt.colorbar(); cb.set_label('elevation difference (m)')